# TP3: Bases de données, analyse et fouille

Ce troisième travail pratique vise à vous faire manipuler, explorer et interpréter des ensembles de données en transports. Vous êtes libres d’utiliser différents outils tels que
* des systèmes de gestion de base de données relationnel (SGBDR comme SQLite, PostgreSQL, Microsoft Access, etc.) ;
* des langages de programmation pour manipuler les données et les visualiser (Matlab, Python, R, etc.) ;
* des outils économétriques (Gretl, Tanagra, R, Python/StatsModels) ;
* des outils de fouille de données (Tanagra,Weka).

## 1. Jeu de données
Les données utilisées sont les temps de parcours collectés par la Ville de Montréal à l’aide de capteurs Bluetooth disponible sur le portail des données ouvertes. Les données sont collectées pour des paires origine-destination pré-définies, décrites dans le fichier des segments routiers de collecte des temps de parcours. Les données de temps de parcours disponibles couvrent la période du 1er janvier 2016 au 28 février 2017. Les attributs des deux fichiers de données sont décrits sur leurs pages respectives.

In [2]:
import pandas as pd
import numpy as np

In [ ]:
seg_routier = pd.read_csv('./donnees/bornes.csv', sep=',', index_col = 0)
trips16 = pd.read_csv('./donnees/trips2016.csv', sep=',', index_col = 0)

## 2. Mandats
### 2.1 Modèle de données
Après analyse des fichiers des segments routiers et des temps de parcours, proposer un modèle de données relationnel pour ces données respectant les trois formes normales (retirer les données redondantes). Indiquer les clefs primaires, externes et les types de données des attributs.

Il n’est pas obligé d’utiliser ce modèle dans la suite des travaux.

In [ ]:
seg_routier

In [4]:
trips16

,SrcDetectorId,DestDetectorId,PathDistance_m,TripStart_dt,TripEnd_dt,Speed_kmh,TravelTime_s
LinkId,,,,,,,
LCC_35-34,35,34,448,2016-01-25 15:40:49,2016-01-25 15:41:16,"59,7333333",27
LCC_35-34,35,34,448,2016-01-25 15:40:52,2016-01-25 15:41:19,"59,7333333",27
LSh_05-06,5,6,403,2016-01-25 15:40:53,2016-01-25 15:41:21,"51,8142857",28
LSh_02-01,2,1,1192,2016-01-25 15:40:19,2016-01-25 15:41:41,"52,3317073",82
LSJ_41-39,41,39,298,2016-01-25 15:40:16,2016-01-25 15:41:45,"12,0539326",89
...,...,...,...,...,...,...,...
LLo_42-41,42,41,1115,2016-01-25 20:23:10,2016-01-25 20:33:00,"6,8033898",590
LSM_14-13,14,13,1753,2016-01-25 20:20:03,2016-01-25 20:33:03,"8,0907692",780
LPi_01-47,1,47,2570,2016-01-25 20:29:50,2016-01-25 20:33:04,"47,6907216",194


## 2.2 Analyse descriptive des temps de parcours
Pour cette première partie, il faut sélectionner deux segments routiers à étudier (le même segment dans les deux directions de déplacement, de A à B et de B vers A) de longueur supérieure à 1500 m. Indiquer clairement les deux segments sélectionnés.

In [ ]:
seg_routier[seg_routier.LineDistance_m>1500]

In [ ]:
#Henri-Bourrassa de Papineau a Lajeunesse
seg_routier.iloc[2]

In [ ]:
#Henri-Bourrassa de Lajeunesse a Papineau
seg_routier.iloc[3]

### Étude de la distribution des temps de parcours
Décrire la distribution des temps de parcours dans une direction, par des statistiques descriptives et des figures. Proposer une méthode pour filtrer les temps de parcours les plus longs si nécessaire. Une fois les données filtrées, faire un test d’adéquation à la loi
normale (ou autre loi théorique plus adaptée).

Le cas échéant, la méthode de filtrage sera appliquée à toutes les données utilisées par
la suite.

In [ ]:
trips16['TravelTime_s'].describe()